In [45]:
include("GeometricSampling.jl");

# Sample points and plot

Here we sample and plot `n=100` points in dimension `d=2` for each of the three gemetries

In [39]:
using Plots

# select plotting backend
plotlyjs()

# number of points and the dimension
n = 100
d = 2

# do the sampling
hyp_pts = sample_hyp(d,n)
euc_pts = sample_euc(d,n)
sph_pts = sample_sph(d,n)

# plot the hyperbolic sample
hyp_plt = scatter(getindex.(hyp_pts,1),getindex.(hyp_pts,2), 
    size=(400,400),  
    title="Hyperbolic sample",
    ylim=(-1.0,1.0),
    xlim=(-1.0,1.0),
    legend=false,
    markersize=1)

# plot the circle of _hyperbolic_ radius 1, or Euclidian radius tanh(1.0/2) within the Poincare disc
x = [tanh(1.0/2)*cos(t) for t=0:0.01:2*pi]
y = [tanh(1.0/2)*sin(t) for t=0:0.01:2*pi]
hyp_plt = plot!(x,y)

# plot the boundary of the Poincare disc
x_1 = [cos(t) for t=0:0.01:2*pi]
y_1 = [sin(t) for t=0:0.01:2*pi]
hyp_plt = plot!(x_1,y_1, linestyle=:dash)

#plot the Euclidian sample
euc_plt = scatter(getindex.(euc_pts,1),getindex.(euc_pts,2), 
    size=(400,400),
    title="Euclidian sample",
    legend=false,
    markersize=1)

# plot the circle of Euclidian radius 1
euc_plt = plot!(x_1,y_1)

# plot the spherical sample
sph_plt = plot(getindex.(sph_pts,1),getindex.(sph_pts,2),getindex.(sph_pts,3), 
    size=(400,400),
    legend=false,
    l=:scatter3d, 
    markersize = 1,
    title="Spherical sample",
    legend=false)

display(hyp_plt)
display(euc_plt)
display(sph_plt)

# Generate samples and compute distances

Now we will generate `maxsim=200` samples for each geometry in each of dimesions `d=[2,4,5,6,8,10]`, compute distance matrices, and save the matrices to `*.jld` files.

In [46]:
using JLD

# number of points in one sample
n = 100

# dimensions
dims = [2,4,5,6,8,10]

# number of samples to generate
maxsim = 2

# curvatures to use
curvs = [-1.0, 0.0, 1.0]

# path where to save the files

path = "data/"

for k in curvs
    for d in dims
        
        # sample points
        pts = [sample_ball(d,n, curvature=k, radius=1.0) for i=1:maxsim]
        
        # compute distances
        D = [distance_matrix(pts[i], curvature=k) for i=1:maxsim]
        
        # conver to density
        dist = pmap(x->to_density(x), D)
        
        # also include the reverse filtration
        dist_r = pmap(x->to_density(-x), D)
        
        # now make the filenames and save the files
        if k < 0.0
            pref = "DIST_hyp_"
        elseif k > 0.0
            pref = "DIST_sph_"
        elseif k == 0.0
            pref = "DIST_euc_"
        end
        
        save(path*pref*"d$(d)_sim$(maxsim)_n$(n).jld", "dist", dist)
        save(path*pref*"r_d$(d)_sim$(maxsim)_n$(n).jld", "bettis", dist_r)
    end
end
        
        
        
        
